## to do

각 hdo 직영 주유소들 기준으로 반경 1km 내의 경쟁사 주유소들과의 게시가 차이를 주차별로 계산

    1. 주유소A를 받아서 반경 1km 내의 경쟁 주유소들의 목록을 뽑아내기
        input args: name, distance
        output: [names] which are different brands
        
    2. 특정 시점에 해당 주유소들의 게시가격 평균 구하기
        input args: date, [names]
        output: (mean(G), mean(D))
        
    3. 주유소A와 반경, 특정 시점을 기준으로 주변 경쟁사들의 평균가격과의 차이를 dict로 뽑기
        input args: name, distance, date
        output: {names: [], date: [], G_diff: [], D_diff: [], lat: [], lon: []}

In [50]:
import pandas as pd
import numpy as np
from haversine import haversine
import folium
import os

In [54]:
df = pd.read_csv('hdo_data_200801.csv')
print(df.shape)
df.head(3)

(50651, 11)


,Unnamed: 0,name,address,date,brand,do_brand,self,G,D,lat,lon
0,0,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 1주,SK에너지,SK에너지,셀프,1238.00,1038.00,37.755264,128.896541
1,1,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 2주,SK에너지,SK에너지,셀프,1273.00,1073.00,37.755264,128.896541
2,2,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 3주,SK에너지,SK에너지,셀프,1290.14,1090.14,37.755264,128.896541


In [55]:
# csv에서 불러왔을 때 생기는 index 제거
df = df.drop('Unnamed: 0', axis=1)
print(df.shape)
df.head(3)

(50651, 10)


,name,address,date,brand,do_brand,self,G,D,lat,lon
0,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 1주,SK에너지,SK에너지,셀프,1238.00,1038.00,37.755264,128.896541
1,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 2주,SK에너지,SK에너지,셀프,1273.00,1073.00,37.755264,128.896541
2,(주)동해에너지주유소,강원 강릉시 경강로 2101(임당동),2020년 06월 3주,SK에너지,SK에너지,셀프,1290.14,1090.14,37.755264,128.896541


In [58]:
t = df.drop_duplicates(['name'])
t.shape

(7561, 10)

In [114]:
t5 = df.drop_duplicates(['address'])
t5.shape

(10112, 10)

In [87]:
class mj_tools:

    def __init__(self, df):
        self.df = df

    def get_near_stations(self, name, distance):

        if name not in list(df['name']):
            raise ValueError ('this station is not in the dataframe')

        self.drop_dup = self.df.drop_duplicates(['name']) # name 기준으로 중복된 row 제거
        self.arr = np.array(self.drop_dup)
        lat = np.array(self.drop_dup['lat'])
        lon = np.array(self.drop_dup['lon'])
        idx_ = list(self.drop_dup['name']).index(name)
        loc_a = ([lat[idx_], lon[idx_]]) # haversine에 넣기 위해 tuple 형태로 변환
        
        list_iter = list(range(len(self.drop_dup)))
        list_iter.remove(idx_) # 본인 index를 제외한 전체 index
        
        result_idx = list()
        final = list()
        
        for i in list_iter:
            loc_b = (lat[i], lon[i])
            if haversine(loc_a, loc_b) < distance:
                result_idx.append(i)
        
        for i in result_idx:
            final.append(self.arr[i][0])
        
        return final
    
    def mean_price(self, year, month, week, names):
        
        self.date_str = '{}년 {:02}월 {}주'.format(year, month, week)
        
        self.df1 = self.df.loc[self.df['date']==self.date_str, ['name', 'G', 'D']]

SyntaxError: invalid syntax (<ipython-input-87-14898ad68c73>, line 34)

In [109]:
name1 = '(주)동해에너지주유소'
name2 = '황제주유소'
date1 = '2020년 06월 1주'

In [105]:
str1 = '{}년 {:02}월 {}주'.format(2020, 6, 2)
str1

'2020년 06월 2주'

In [111]:
t1 = df.loc[df['date']==str1]
#t2 = t1.loc[t1['name'] in ]

(10073, 10)

7502

In [82]:
t = mj_tools(df)

In [86]:
t.get_near_stations('(주)동해에너지주유소', 1)

['삼거리주유소', '홍길동주유소', '화부산주유소']

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50651 entries, 0 to 50650
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      50651 non-null  object 
 1   address   50651 non-null  object 
 2   date      50651 non-null  object 
 3   brand     50651 non-null  object 
 4   do_brand  50651 non-null  object 
 5   self      50651 non-null  object 
 6   G         50651 non-null  float64
 7   D         50651 non-null  float64
 8   lat       49294 non-null  float64
 9   lon       49294 non-null  float64
dtypes: float64(4), object(6)
memory usage: 3.9+ MB
